In [13]:
from dotenv import load_dotenv
import os
from agents import Agent, Runner, trace, function_tool
import asyncio
import sendgrid
from IPython.display import display, Markdown
from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import List, Dict
import plantuml
import json

In [14]:
# Agent Instructions
INSTRUCTION_JUNIOR_DEVELOPER = """You are a Junior Developer tasked with creating the initial system design for a car sharing app.

Your responsibilities:
1. System Analysis:
   - Document core user flows and features
   - Identify key technical requirements
   - List potential technical challenges

2. Initial Design Documentation:
   - Create high-level system architecture diagram
   - Design basic database schema (focus on core entities)
   - Draft API endpoints for main features
   - Propose initial technology stack

3. UML Diagrams (focus on):
   - Use Case diagram for core features
   - Basic Class diagram for main entities
   - Sequence diagram for booking flow
   - Entity-Relationship diagram for database

Key Features to Consider:
- User authentication and authorization
- Car inventory and location management
- Booking and reservation system
- Trip management and status tracking
- Payment processing
- Real-time car location tracking

Please provide clear explanations for your choices and highlight any areas where you need senior guidance."""

INSTRUCTION_SENIOR_DEVELOPER = """You are a Senior Developer responsible for reviewing and enhancing the junior developer's system design for the car sharing app.

Your responsibilities:
1. Technical Review:
   - Evaluate proposed architecture for scalability and reliability
   - Assess technology stack choices
   - Review database design for performance and data integrity
   - Analyze API design for security and efficiency

2. Enhancement Proposals:
   - Suggest improvements for handling high concurrency
   - Recommend caching strategies
   - Propose solutions for real-time updates
   - Address potential security concerns
   - Suggest monitoring and logging solutions

3. Task Breakdown:
   - Create detailed epics and user stories
   - Estimate story points (1,2,3,5,8,13)
   - Identify technical dependencies
   - Highlight potential risks and mitigation strategies

4. Performance Considerations:
   - Load balancing strategies
   - Database optimization techniques
   - API response time optimization
   - Mobile app performance guidelines

Please provide detailed explanations for your recommendations and include specific examples where possible."""

INSTRUCTION_TEAM_LEADER = """You are a Tech Leader responsible for final review, refinement, and communication of the car sharing app system design.

Your responsibilities:
1. Strategic Review:
   - Ensure alignment with business requirements
   - Validate architectural decisions
   - Verify scalability approach
   - Confirm security measures
   - Review cost implications

2. Documentation Quality:
   - Ensure clarity and completeness
   - Verify technical accuracy
   - Check consistency across all documents
   - Validate terminology usage

3. Implementation Strategy:
   - Define development phases
   - Establish technical milestones
   - Set quality metrics
   - Define success criteria
   - Plan for monitoring and maintenance

4. Risk Assessment:
   - Technical debt evaluation
   - Resource requirements
   - Timeline feasibility
   - Third-party dependencies
   - Compliance requirements

5. Final Deliverables:
   - Executive summary
   - Architecture decision records
   - Implementation roadmap
   - Resource allocation plan
   - Budget estimates

6. Email Communication:
   - Structure the email with clear sections using HTML
   - Use professional formatting with:
     * Consistent heading hierarchy (h1, h2, h3)
     * Well-organized bullet points and lists
     * Proper spacing between sections
     * Highlighted key points and decisions
     * Professional color scheme (navy, dark gray for text)
     * Clean, readable font (Arial/Helvetica)
   - Include:
     * Clear executive summary at the top
     * Organized technical details in collapsible sections
     * Visual separation between different components
     * Professional footer with contact information
     * Clear next steps and action items

Please provide actionable feedback and ensure all documentation is product-owner friendly while maintaining technical accuracy. Format the email professionally to enhance readability and highlight key information."""

INSTRUCTION_UML_DESIGNER = """You are a UML Design Expert specialized in creating PlantUML diagrams for software systems.

Your responsibilities:
1. Create clear and accurate UML diagrams using PlantUML syntax for:
   - Use Case Diagrams
   - Class Diagrams
   - Sequence Diagrams
   - Entity-Relationship Diagrams

2. Follow these guidelines:
   - Use proper PlantUML notation and syntax
   - Include clear relationships between entities
   - Add meaningful descriptions and notes where necessary
   - Use appropriate stereotypes and icons
   - Maintain consistent naming conventions

Please provide the PlantUML code with clear sections and comments."""

In [15]:
@function_tool
def send_email_to_product_owner(subject: str, html_body: str) -> Dict[str, str]:
    """Send an email to the product owner with the technical analysis and plans"""
    # Add this debug code to check your environment variables
    print("EMAIL_FROM:", os.environ.get('EMAIL_FROM'))
    print("EMAIL_TO:", os.environ.get('EMAIL_TO', 'trungnv184@gmail.com'))
    print("SENDGRID_API_KEY:", "exists" if os.environ.get('SENDGRID_API_KEY') else "missing")
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email(os.environ.get('EMAIL_FROM'))
    to_email = To(os.environ.get('EMAIL_TO', 'trungnv184@gmail.com'))
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content)
    
    response = sg.client.mail.send.post(request_body=mail.get())
    return {"status": "success"}
    


In [16]:
# Initialize Agents
agent_junior_developer = Agent(
    name="Junior Developer Agent",
    instructions=INSTRUCTION_JUNIOR_DEVELOPER,
    model="gpt-4-turbo-preview"
)

agent_senior_developer = Agent(
    name="Senior Developer Agent",
    instructions=INSTRUCTION_SENIOR_DEVELOPER,
    model="gpt-4-turbo-preview"
)

# agent_team_leader = Agent(
#     name="Team Leader Agent",
#     instructions=INSTRUCTION_TEAM_LEADER,
#     model="gpt-4-turbo-preview"
# )
# Add the email tool to your agent if not already done
agent_team_leader = Agent(
    name="Team Leader Agent",
    instructions=INSTRUCTION_TEAM_LEADER,
    tools=[send_email_to_product_owner],  # Add the email tool here
    model="gpt-4-turbo-preview"
)

uml_designer = Agent(
    name="UML Designer",
    instructions=INSTRUCTION_UML_DESIGNER,
    model="gpt-4-turbo-preview"
)



In [17]:
def parse_uml_diagrams(output: str) -> Dict[str, str]:
    """Parse the UML designer's output into separate diagrams"""
    diagrams = {}
    current_diagram = None
    current_content = []
    
    for line in output.split('\n'):
        if line.startswith('### ') and 'Diagram' in line:
            if current_diagram:
                diagrams[current_diagram] = '\n'.join(current_content)
            current_diagram = line.replace('### ', '').strip()
            current_content = []
        elif current_diagram:
            current_content.append(line)
    
    if current_diagram:
        diagrams[current_diagram] = '\n'.join(current_content)
    
    return diagrams

def save_plantuml(diagram_type: str, content: str) -> Dict[str, str]:
    """Save PlantUML diagram content to a file"""
    print(diagram_type)
    filename = f"diagrams/{diagram_type.lower().replace(' ', '_')}.puml"
    os.makedirs("diagrams", exist_ok=True)
    
    with open(filename, "w") as f:
        f.write(content)
    
    return {"status": "success", "filename": filename}


def generate_html_diagrams(diagrams: Dict[str, str]) -> str:
    """Convert PlantUML diagrams to HTML format"""
    html = ""
    for diagram_type, content in diagrams.items():
        html += f"<h3>{diagram_type}</h3>\n"
        html += f"<pre><code>\n{content}\n</code></pre>\n"
    return html

In [18]:

async def junior_developer_analysis(message: str):
    # Step 1: Initial analysis and proposal
    with trace("Junior Developer Initial Analysis"):
        initial_analysis = await Runner.run(agent_junior_developer, message)
        
    # Step 2: Generate UML diagrams
    with trace("UML Diagram Generation"):
        uml_result = await Runner.run(uml_designer, """
        Please create the following UML diagrams for the car sharing app:
        1. Use Case Diagram showing core features
        2. Class Diagram showing main entities and their relationships
        3. Sequence Diagram for the booking flow
        4. ER Diagram for the database schema
        """)
        
        # Save and process UML diagrams
        diagrams = parse_uml_diagrams(uml_result.final_output)
        for diagram_type, content in diagrams.items():
            save_plantuml(diagram_type, content)
            
    return {
        "analysis": initial_analysis.final_output,
        "diagrams": diagrams
    }

async def senior_developer_review(junior_work: dict):
    review_message = f"""
    Please review the junior developer's work and provide detailed feedback:
    
    Initial Analysis and Design:
    {junior_work['analysis']}
    
    UML Diagrams:
    {json.dumps(junior_work['diagrams'], indent=2)}
    
    Focus on:
    1. Architecture scalability and reliability
    2. Technology stack appropriateness
    3. Database design optimization
    4. Security considerations
    5. Performance improvements
    """
    
    with trace("Senior Developer Review"):
        review_result = await Runner.run(agent_senior_developer, review_message)
    return review_result.final_output

async def team_leader_review(junior_work: dict, senior_review: str):
    review_message = f"""
    Please review the complete work from the team and provide:
    1. Final technical analysis
    2. Detailed task breakdown
    3. Implementation strategy
    4. Risk assessment
    5. Resource planning
    
    Junior Developer's Work:
    {junior_work['analysis']}
    
    Senior Developer's Review:
    {senior_review}
    """
    
    with trace("Team Leader Review"):
        review_result = await Runner.run(agent_team_leader, review_message)
    return review_result.final_output


# async def prepare_and_send_email(junior_work: dict, senior_review: str, team_leader_review: str):
#     email_content = f"""
#     <h1>Car Sharing App Technical Analysis and Implementation Plan</h1>
    
#     <h2>Executive Summary</h2>
#     {team_leader_review}
    
#     <h2>Detailed Technical Analysis</h2>
#     <h3>Initial Design (Junior Developer)</h3>
#     {junior_work['analysis']}
    
#     <h3>Technical Review (Senior Developer)</h3>
#     {senior_review}
    
#     <h2>UML Diagrams</h2>
#     {generate_html_diagrams(junior_work['diagrams'])}
#     """
    
#     # Create a Runner instance to run the email sending function
#     result = await Runner.run(
#         agent_team_leader,  # You can use any of your agents here
#         f"""Please send this email:
#         Subject: Car Sharing App - Technical Analysis and Implementation Plan
#         Content: {email_content}
#         """
#     )
    
#     return {"status": "success", "message": "Email sent successfully"}
async def prepare_and_send_email(junior_work: dict, senior_review: str, team_leader_review: str):
    email_format_request = f"""
    Please format the following technical analysis into a professional HTML email with clear sections, proper styling, and visual hierarchy:

    Team Leader Review:
    {team_leader_review}

    Initial Analysis:
    {junior_work['analysis']}

    Senior Review:
    {senior_review}

    UML Diagrams:
    {json.dumps(junior_work['diagrams'], indent=2)}

    Requirements:
    - Use professional HTML formatting
    - Include clear section headings
    - Highlight key points
    - Ensure readability
    - Add proper spacing
    - Include a professional footer
    """
    
    with trace("Send Email to Product Owner"):
        # Format the email content using the email_format_request
        formatted_email = await Runner.run(
            agent_team_leader,
            email_format_request
        )
        
        # Send the formatted email
        result = await Runner.run(
            agent_team_leader,
            f"Please send this technical analysis email with subject: Car Sharing App - Technical Analysis and Implementation Plan\n\nContent:\n{formatted_email}"
        )

    return {"status": "success", "message": "Email sent successfully", "details": result}

async def run_car_sharing_app_development():
    # Step 1: Junior Developer Analysis and UML Generation
    print("Step 1: Junior Developer Analysis and UML Generation...")
    junior_work = await junior_developer_analysis(
        "Please analyze and create initial system design for the car sharing app"
    )
    
    # Step 2: Senior Developer Review
    print("\nStep 2: Senior Developer Review...")
    senior_review = await senior_developer_review(junior_work)
    
    # Step 3: Team Leader Review
    print("\nStep 3: Team Leader Review...")
    team_leader_result = await team_leader_review(junior_work, senior_review)
    
    # Step 4: Prepare and Send Email
    print("\nStep 4: Preparing and Sending Email to Product Owner...")
    email_result = await prepare_and_send_email(
        junior_work,
        senior_review,
        team_leader_result
    )
    
    print("\nProcess completed!")
    return {
        "junior_work": junior_work,
        "senior_review": senior_review,
        "team_leader_review": team_leader_result,
        "email_status": email_result
    }

# Run the process
results = await run_car_sharing_app_development()

1. Use Case Diagram for Core Features
2. Class Diagram for Main Entities
3. Sequence Diagram for the Booking Flow
4. ER Diagram for the Database Schema

Step 2: Senior Developer Review...

Step 3: Team Leader Review...

Step 4: Preparing and Sending Email to Product Owner...
EMAIL_FROM: andynguyenvn2023@gmail.com
EMAIL_TO: trungnv184@gmail.com
SENDGRID_API_KEY: exists
EMAIL_FROM: andynguyenvn2023@gmail.com
EMAIL_TO: trungnv184@gmail.com
SENDGRID_API_KEY: exists

Process completed!
